# TUTORIAL: Real-time data assimilation in a chaotic system
<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.4/MathJax.js?config=default'></script>


* [The Ensemble Kalman filter (recap)](#enKF)
* [Test case: twin experiment on the Lorenz63 model](#twin)
    * [The true state and parameters](#truth)
    * [The observations](#obs)
    * [The ensemble](#ens)
    * [Data assimilation](#da)

In [4]:
import os, sys
module_path = os.path.abspath(os.path.join('funcs'))
if module_path not in sys.path:
    sys.path.append(module_path)

## The Ensemble Kalman filter <a class="anchor" id="enKF"></a>

As explained in tutorial 10, real-time data assimilation updates our knowledge in the model state $\boldsymbol{\phi}$ and the parameters $\boldsymbol{\alpha}$ when observation data of the modelled physical quantity become available. To perform state and parameter estimation, we introduced the augmented state vector $\boldsymbol{\psi}=[\boldsymbol{\phi};\boldsymbol{\alpha};\mathbf{q}]$, such that
$$
\begin{align}
\left\{
\begin{array}{rcl}
\mathrm{d}\boldsymbol{\psi} &=& \mathbf{F}\left(\boldsymbol{\psi} +\boldsymbol{\epsilon}_\psi\right){\mathrm{d} t}  \\
\mathbf{q} &=& \mathbf{M}\mathbf{\psi}  + \boldsymbol{\epsilon}_{q}
\end{array}
\right.
\end{align}
$$ where $\mathbf{F}$ and $\boldsymbol{\epsilon}_\psi$ are the augmented nonlinear operator and aleatoric uncertainties, respectively; $\mathbf{M} = \left[\mathbf{0}~\big|~\mathbb{I}_{N_q}\right]$ is the linear measurement operator.

We take a stochastic ensemble data assimilation approach to tackle nonlinear dynamics. Algorithmically, 
1. we forecast in time $m$-realizations of the augmented state to obtain an ensemble of forecast states $\boldsymbol{\psi}_j^\text{f}$; 
2. when data $\mathbf{d}$ become available, we use real-time data assimilation (e.g., the ensemble Kalman filter (EnKF)) to statistically combine the data with the forecast ensemble, which yields the analysis ensemble $\boldsymbol{\psi}_j^\mathrm{a}$; 
3. we re-initialize the model with the analysis ensemble to perform the next analysis step. 

</p>


In [5]:
%matplotlib widget

import numpy as np
from scipy import linalg
rng = np.random.default_rng(0)

def EnKF(case, observations, C_dd):
    """ 
        Inputs:
            case: ensemble forecast as a class object
            observations: observation at time t
            C_dd: observation error covariance matrix
        Returns:
            Aa: analysis ensemble 
    """
    Af = np.vstack((case.get_current_state, 
                    case.get_observables()))    # augmented state matrix [phi; alpha; q] x m
    
    psi_f_m = np.mean(Af, 1, keepdims=True)     # ensemble mean
    Psi_f = Af - psi_f_m                        # deviations from the mean
    
    # Create an ensemble of observations
    D = rng.multivariate_normal(observations, C_dd, case.m).transpose()
    
    # Mapped forecast matrix M(Af) and mapped deviations M(Af')
    Y = np.dot(case.M, Af)
    S = np.dot(case.M, Psi_f)
    
    C = (case.m - 1) * Cdd + np.dot(S, S.T)  # Kalman gain matrix
    X = np.dot(S.T, np.dot(linalg.inv(C), (D - Y)))

    return Af + np.dot(Af, X)



## Test case: twin experiment on the Lorenz63 model <a class="anchor" id="twin"></a>


In real scenarios, we do not have access to the truth, and thus we do not know which are the optimal state and parameters which estimate the system under investigation. In this tutorial, we investigate the EnKF with a twin experiment, which consist of generating a synthetic true state from the same numerical model as the forecast, and then create the observations by adding stochastic uncertainty to the synthetic truth. This is, we compute the truth $\mathbf{d}^\dagger$ as 
$$
\begin{equation}
\left\{
\begin{array}{rcl}
\mathrm{d}\boldsymbol{\psi^\dagger } &=& \mathbf{F}\left(\boldsymbol{\psi}^\dagger\right){\mathrm{d} t}  \\
\mathbf{d}^\dagger &=& \mathbf{M}\mathbf{\psi}^\dagger
\end{array}
\right.
\end{equation}
$$
and the observations as  $\mathbf{d} = \mathbf{d}^\dagger +  \boldsymbol{\epsilon}_d$. During the assimilation we assume that we do not know the true state and parameters $\mathbf{\psi}^\dagger$, and our objective is to recover them. 

We investigate the Lorenz63 model consists of the system of ordinary differential equations 
$$
\begin{align}
\left\{
\begin{array}{rcl}
\frac{\mathrm{d}x}{\mathrm{d}t} &= &\sigma (y - x) \\
\frac{\mathrm{d}y}{\mathrm{d}t} &= &x (\rho - z) - y \\
\frac{\mathrm{d}z}{\mathrm{d}t} &= &x y - \beta z.
\end{array}\right.
\end{align}
$$

In the state-space formulation introduced above, we have the state variables $\boldsymbol{\phi}=[x;y;x]$, the model parameters  $\boldsymbol{\alpha}=[\sigma;\rho;\beta]$ and the operator $\mathbf{F}$ is the right hand side of the equations. In this problem, we assume that the observables are the state variables. 
 
### Chaotic dynamics in data assimilation

The Lorenz63 model exhibits chaotic dynamics at certain combinations of parameters. Chaotic systems make data assimilation challenging because their predictability horizon is short due to the butterfly effect. For instance, at $\boldsymbol{\alpha}=[10; 28; 8/3]$ two solutions initialized with 0.01\% difference evolve as 

<img src="figs/Lorenz_butterfly.gif" width="750" align="center">

The predictability horizon in chaotic systems is given by the Lyapunov time, i.e., the time for the distance between two nearby trajectories in the phase space increases by a factor of e. The Lyapunov time is th inverse of the maximal Lyapunov exponent. For the Lorenz system, the maximal Lyapunov exponent is known to be about 0.9056. 



### 1) The true state and parameters <a class="anchor" id="truth"></a>



In [6]:

from essentials.physical_models import Lorenz63


dt_t = 0.015
t_lyap = 0.9056 ** (-1)  # Lyapunov Time (inverse of maximal Lyapunov exponent)

true_params = dict(dt=dt_t,
                   rho=28.,
                   sigma=10.,
                   beta=8. / 3.,
                   psi0=rng.random(3)+10,
                   observe_dims=[0, 1]     # Select the dimensions to observe
                   )

# Initialize model
true_case = Lorenz63(**true_params)

# Forecast model
t_max = t_lyap * 100
psi, t = true_case.time_integrate(int(t_max / true_case.dt))
true_case.update_history(psi, t)


### 2) The observations <a class="anchor" id="obs"></a>

In [7]:
from essentials.create import create_noisy_signal

# Retrieve observables
d_true = true_case.get_observable_hist().squeeze()
t_true = true_case.hist_t

std_obs = 0.02
noise_type = 'gaussian, additive'  # gaussian or coloured noise (white, pink, brown, blue, and violet), and either additive or multiplicative. 
d_raw = create_noisy_signal(d_true, noise_level=std_obs, noise_type=noise_type)


# Draw data points from raw data
dt_obs = t_lyap * .4  # time between analyses
Nt_obs = dt_obs // dt_t
t_start, t_stop = t_lyap * 10, t_lyap * 60 # start and end of assimilation

obs_idx = np.arange(t_start // dt_t, t_stop // dt_t + Nt_obs, Nt_obs, dtype=int)
d_obs, t_obs = d_raw[obs_idx].squeeze(), t_true[obs_idx]


In [8]:
# Visualize the truth and observations
from essentials.plotResults import plot_truth
plot_truth(model=Lorenz63, y_raw=d_raw, y_true=d_true, t=t_true, dt=dt_t, y_obs=d_obs, t_obs=t_obs, f_max=10)

### 3) Define the forecast ensemble  <a class="anchor" id="ens"></a>
We create an ensemble with randomly generated states and parameters

In [9]:
from essentials.create import create_ensemble

alpha0 = dict(rho=(25., 35.),
              beta=(2, 4),
              sigma=(5,15))

forecast_params = dict(model=Lorenz63,      
                       dt=dt_t,             
                       m=50,               # Number of ensemble members
                       std_psi=0.2,        # Initial uncertainty in the state
                       std_a=alpha0,       # Initial uncertainty in the parameters
                       sigma=10.,
                       beta=8. / 3.,
                       observe_dims=true_params['observe_dims']
                       )

ensemble = create_ensemble(**forecast_params)


In [10]:
# Visualize ensemble initialization
from essentials.plotResults import plot_ensemble
plot_ensemble(ensemble)

### 4) Apply data assimilation <a class="anchor" id="da"></a>
We now have all the ingredients to start our data assimilation algorithm.

In [11]:

filter_ens = ensemble.copy()

# Define measurement error covariance matrix
Cdd = np.diag((1. * np.ones(filter_ens.Nq))) ** 2

# ----------------------------------------------------------------------------
for d, t_d in zip(d_obs, t_obs):
    # Parallel forecast to next observation
    Nt = int(np.round((t_d - filter_ens.get_current_time) / filter_ens.dt))
    psi, t = filter_ens.time_integrate(Nt)
    filter_ens.update_history(psi, t)
    # Perform assimilation 
    Aa = EnKF(filter_ens, d, Cdd) 
    # Update the initial condition for the next forecast
    filter_ens.update_history(Aa[:-filter_ens.Nq, :], update_last_state=True)
# ----------------------------------------------------------------------------

filter_ens.close() # closes the parallel pools


In [12]:
from essentials.plotResults import plot_timeseries, plot_parameters
true_args = dict(y_raw=d_raw, y_true=d_true, t=t_true, dt=dt_t, t_obs=t_obs, y_obs=d_obs, dt_obs=Nt_obs * dt_t)

ens = filter_ens.copy()

#Forecast the ensemble further without assimilation
Nt_extra = int(10 * t_lyap / ens.dt) 

psi, t = ens.time_integrate(Nt_extra, averaged=False)
ens.update_history(psi, t)

plot_timeseries(ens, true_args, reference_t=t_lyap, plot_ensemble_members=True)
plot_parameters(ens, true_args, reference_p=true_params)

In [13]:
# Visualize attractors
from essentials.plotResults import plot_attractor

case0 = true_case.copy()
case1 = filter_ens.copy()

# Forecast both cases
Nt = 20 * int(t_lyap / ens.dt)
psi0, t0 = case0.time_integrate(Nt=Nt)
psi1, t1 = case1.time_integrate(Nt=Nt, averaged=True)

plot_attractor([psi0, psi1], color=['gray', 'teal'])




<br />

___

### Exercise:

<em> <font color=#7CB9E8> -How would you expect the solution to change if we decrease/increase the assimilation frequency?


<em> <font color=#7CB9E8> -How would you expect the solution to change if we observe only part of the state?
</em>


<br />

___
